In [22]:
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pandas_datareader.data as web
import glob
import os


In [23]:
# Quick look at one CSV to confirm structure
sample_path = sorted(glob.glob('*.csv'))[0]
sample_df = pd.read_csv(sample_path, header=None)
sample_df.head()

,0,1
0,2015.003164,0.105111
1,2015.045696,0.120294
2,2015.045696,0.097520
3,2015.088229,0.120294
4,2015.088229,0.097520


In [24]:
MONTHS = {
    'JAN': 1, 'FEB': 2, 'MAR': 3, 'APR': 4, 'MAY': 5, 'JUN': 6,
    'JUL': 7, 'AUG': 8, 'SEP': 9, 'OCT': 10, 'NOV': 11, 'DEC': 12
}

def date_to_decimal_year(dt):
    start = datetime(dt.year, 1, 1)
    end = datetime(dt.year + 1, 1, 1)
    return dt.year + (dt - start).total_seconds() / (end - start).total_seconds()

def decimal_year_to_date(x):
    year = int(np.floor(x))
    frac = x - year
    start = datetime(year, 1, 1)
    end = datetime(year + 1, 1, 1)
    return start + (end - start) * frac

def parse_forecast_date_from_filename(filename):
    base = os.path.basename(filename)
    m = re.match(r'^([A-Za-z]{3})(\d{2})FFR\.csv$', base)
    if not m:
        raise ValueError(f'Unrecognized forecast filename: {base}')
    tag, year_two = m.group(1).upper(), int(m.group(2))
    year = 2000 + year_two
    month = MONTHS[tag]
    return datetime(year, month, 1)

def rdp_indices(x, y, eps):
    pts = np.column_stack([x, y])
    n = len(pts)
    if n <= 2:
        return np.arange(n)

    keep = np.zeros(n, dtype=bool)
    keep[0] = True
    keep[-1] = True
    stack = [(0, n - 1)]

    while stack:
        i, j = stack.pop()
        if j <= i + 1:
            continue
        a = pts[i]
        b = pts[j]
        ab = b - a
        ab2 = ab @ ab
        if ab2 == 0:
            d = np.linalg.norm(pts[i + 1:j] - a, axis=1)
        else:
            ap = pts[i + 1:j] - a
            t = (ap @ ab) / ab2
            proj = a + np.outer(t, ab)
            d = np.linalg.norm(pts[i + 1:j] - proj, axis=1)
        k_rel = int(np.argmax(d))
        dmax = float(d[k_rel]) if len(d) else 0.0
        k = i + 1 + k_rel
        if dmax > eps:
            keep[k] = True
            stack.append((i, k))
            stack.append((k, j))
    return np.where(keep)[0]

def quarter_bounds(dt):
    q = (dt.month - 1) // 3
    start = datetime(dt.year, q * 3 + 1, 1)
    end = start + relativedelta(months=3)
    return start, end

def avg_over_interval(x_s, y_s, start_x, end_x):
    if start_x < x_s[0] or end_x > x_s[-1] or end_x <= start_x:
        return np.nan
    total = 0.0
    for i in range(len(x_s) - 1):
        seg_start = x_s[i]
        seg_end = x_s[i + 1]
        if seg_end <= start_x or seg_start >= end_x:
            continue
        a = max(start_x, seg_start)
        b = min(end_x, seg_end)
        y0 = y_s[i]
        y1 = y_s[i + 1]
        slope = (y1 - y0) / (seg_end - seg_start)
        total += y0 * (b - a) + slope * 0.5 * ((b - seg_start) ** 2 - (a - seg_start) ** 2)
    return total / (end_x - start_x)

def fred_avg(start_dt, end_dt):
    if end_dt < start_dt:
        return np.nan
    df = web.DataReader('DFF', 'fred', start_dt, end_dt)
    return float(df['DFF'].mean()) if not df.empty else np.nan


In [25]:
def process_csv(path, H=12, x_round=3, eps=0.015):
    raw = pd.read_csv(path, header=None, names=['x', 'y'])
    raw = raw.apply(pd.to_numeric, errors='coerce').dropna()

    raw['x_bin'] = raw['x'].round(x_round)

    def midline_agg(s):
        spread = s.max() - s.min()
        return s.min() if spread > 0.07 else s.median()

    mid = raw.groupby('x_bin')['y'].apply(midline_agg).reset_index().rename(
        columns={'x_bin': 'x', 'y': 'y_mid'}
    ).sort_values('x')

    x = mid['x'].to_numpy()
    y = mid['y_mid'].to_numpy()
    idx = rdp_indices(x, y, eps=eps)
    x_s = x[idx]
    y_s = y[idx]

    forecast_date = parse_forecast_date_from_filename(path)
    q_start, q_end = quarter_bounds(forecast_date)

    prev_start = q_start - relativedelta(months=3)
    prev_end = q_start - relativedelta(days=1)
    lffr = fred_avg(prev_start, prev_end)

    first_x = x_s[0]
    last_x = x_s[-1]
    first_dt = decimal_year_to_date(first_x)

    out = {}
    for h in range(H + 1):
        start = q_start + relativedelta(months=3 * h)
        end = q_start + relativedelta(months=3 * (h + 1))
        start_x = date_to_decimal_year(start)
        end_x = date_to_decimal_year(end)

        if end_x > last_x:
            out[f'ffr_{h}'] = np.nan
            continue

        if h == 0 and start_x < first_x < end_x:
            fred_end = pd.Timestamp(first_dt).normalize()
            dat = fred_avg(start, fred_end.to_pydatetime())
            interp = avg_over_interval(x_s, y_s, first_x, end_x)
            if np.isnan(dat) or np.isnan(interp):
                out[f'ffr_{h}'] = np.nan
                continue
            q_prop = (first_x - start_x) / (end_x - start_x)
            out[f'ffr_{h}'] = q_prop * dat + (1 - q_prop) * interp
        else:
            out[f'ffr_{h}'] = avg_over_interval(x_s, y_s, start_x, end_x)

    out_row = {'Date': forecast_date}
    out_row['lffr'] = lffr
    out_row.update(out)
    return out_row


In [26]:
H = 12
rows = []
pattern = re.compile(r'^[A-Za-z]{3}\d{2}FFR\.csv$')
for path in sorted(glob.glob('*.csv')):
    if not pattern.match(path):
        continue
    rows.append(process_csv(path, H=H))

df_out = pd.DataFrame(rows).sort_values('Date').reset_index(drop=True)
df_out = df_out.dropna(axis=1, how='all')

df_out.to_csv('ffr_quarterly_forecasts.csv', index=False)
df_out.head()


,Date,lffr,ffr_0,ffr_1,ffr_2,ffr_3,ffr_4,ffr_5,ffr_6,ffr_7,ffr_8,ffr_9,ffr_10,ffr_11,ffr_12
0,2015-01-01,0.101304,0.112704,0.186917,0.469325,0.820834,1.176844,1.501327,1.811605,2.126064,2.421171,2.704726,2.951103,NaN,NaN
1,2015-03-01,0.101304,0.125582,0.184167,0.403892,0.666044,0.954400,1.244722,1.505795,1.773618,2.004318,2.249474,2.478482,NaN,NaN
2,2015-04-01,0.112889,0.129791,0.165464,0.376902,0.646176,0.904903,1.133261,1.386009,1.611275,1.854972,2.076822,NaN,NaN,NaN
3,2015-06-01,0.112889,0.135438,0.163201,0.351262,0.590063,0.824719,1.031571,1.263089,1.476983,1.691120,1.908482,NaN,NaN,NaN
4,2015-07-01,0.125604,0.169212,0.347553,0.582795,0.804029,1.012605,1.244575,1.443000,1.640089,1.841454,NaN,NaN,NaN,NaN


In [27]:
# Check for duplicate x values and report max |y diff| per file
for path in sorted(glob.glob('*.csv')):
    df = pd.read_csv(path, header=None, names=['x', 'y']).apply(pd.to_numeric, errors='coerce').dropna()
    dup = df[df.duplicated(subset=['x'], keep=False)]
    if dup.empty:
        continue
    diff_by_x = dup.groupby('x')['y'].agg(lambda s: s.max() - s.min())
    max_x = diff_by_x.idxmax()
    max_diff = diff_by_x.loc[max_x]
    print(f"{path}: max |y diff| = {max_diff} at x = {max_x}")


APR15FFR.csv: max |y diff| = 0.0303664045507777 at x = 2015.30089283622
APR16FFR.csv: max |y diff| = 0.06149024922184321 at x = 2017.311813157728
APR17FFR.csv: max |y diff| = 0.05663144319174673 at x = 2018.4996903811752
APR18FFR.csv: max |y diff| = 0.05993808049535598 at x = 2018.613074792244
APR19FFR.csv: max |y diff| = 0.022937905468026543 at x = 2021.2884615384612
DEC15FFR.csv: max |y diff| = 0.030180208268379083 at x = 2015.8881600229088
DEC16FFR.csv: max |y diff| = 0.05703959773727174 at x = 2017.1764386395964
DEC17FFR.csv: max |y diff| = 0.03036640455077899 at x = 2018.3420603723207
DEC18FFR.csv: max |y diff| = 0.09855889724310796 at x = 2019.0252155813264
DEC19FFR.csv: max |y diff| = 0.022652577276698338 at x = 2022.5581794361945
JAN16FFR.csv: max |y diff| = 0.056543230389000776 at x = 2016.3754860093056
JAN17FFR.csv: max |y diff| = 0.0609520611199732 at x = 2017.601100223483
JAN18FFR.csv: max |y diff| = 0.057889919707117654 at x = 2018.439283694977
JAN19FFR.csv: max |y diff| =